In [1]:
# Reduce el consumo por cada municipio
# a una tarifa tipo 1 y DAC.

import pandas as pd
import numpy as np

In [7]:
path = "../../../datos/CFE/"
fname = "Consumo_electrico_2010_2017.csv"

# Carga el archivo y elimina el 2017.
df = ( pd.read_csv(path + fname)
    .drop( ["Consumo 2017", "Usuarios 2017"],
    axis = 1 ) )
# Elimina espacios excedentes.
df["Tarifa"] = df["Tarifa"].str.strip()
# Convierte valores negativos en positivos.
df.iloc[:,6:] = df.iloc[:,6:].abs()
# Quita resúmenes estatales y por país.
df = df.query('CVE_MUN!=999 and CVE_ENT!=999 ')
# Escoge solo las tarifas residenciales.
df = df.query(
    'Tarifa=="1" or Tarifa=="1A" or Tarifa=="1B" '
    'or Tarifa=="1C" or Tarifa=="1D" or Tarifa=="1E" '
    'or Tarifa=="1F"'
    ' or Tarifa=="DAC"'
    )
# Quita la tarifa DAC.
df_2 = df.where(df["Tarifa"] != "DAC").dropna()

In [8]:
# Cálculos con tarifa DAC.
df_per = df.copy()

# Promedia todos los años y 
# quita el valor de cada año.
df_per["Consumo"] = ( df_per.iloc[:,6:13]
    .mean(axis = 1) )
df_per.drop(df_per.columns[6:13],
    axis = 1, inplace = True)
df_per["Usuarios"] = ( df_per.iloc[:,6:13]
    .mean(axis = 1) )
df_per.drop(df_per.columns[6:13],
    axis = 1, inplace = True)

# Suma el total por municipio.
df_sum = ( df_per.groupby("CVE_INEGI").sum()
    .drop(df_per.columns[0:2], axis = 1) )

# Obtiene el porcentaje
# correspondiente a cada tarifa.
df_dummy = df_per.copy()
for i in df_per.itertuples():
    df_dummy.loc[i[0], "Consumo"] = ( i[7] /
        ( df_sum.iloc[df_sum.index == i[3]]
        .iloc[0,0] ) )
    df_dummy.loc[i[0], "Usuarios"] = ( i[8] /
        ( df_sum.iloc[df_sum.index == i[3]]
        .iloc[0,1] ) )

# Cálculos sin tarifa DAC.
df_per = df_dummy.copy()

# Promedia todos los años y 
# quita el valor de cada año.
df_per2 = df_2.copy()
df_per2["Consumo"] = ( df_per2.iloc[:,6:13]
    .mean(axis = 1) )
df_per2.drop(df_per2.columns[6:13],
    axis = 1, inplace = True)
df_per2["Usuarios"] = ( df_per2.iloc[:,6:13]
    .mean(axis = 1) )
df_per2.drop(df_per2.columns[6:13],
    axis = 1, inplace = True)

# Suma el total por municipio.
df_sum2 = ( df_per2.groupby("CVE_INEGI").sum()
    .drop(df_per.columns[0:2], axis = 1) )

# Obtiene el porcentaje
# correspondiente a cada tarifa.
df_dummy = df_per2.copy()
for i in df_per2.itertuples():
    df_dummy.loc[i[0], "Consumo"] = ( i[7] /
        ( df_sum.iloc[df_sum.index == i[3]]
        .iloc[0,0] ) )
    df_dummy.loc[i[0], "Usuarios"] = ( i[8] /
        ( df_sum.iloc[df_sum.index == i[3]]
        .iloc[0,1] ) )

df_per2 = df_dummy.copy()

In [9]:
# Obtiene la tarifa que corresponde
# al máximo con☼umo por municipio.
max_cons = ( df_per2[["CVE_INEGI","Consumo"]]
    .groupby("CVE_INEGI").max() )

# Pone uno en Bool a la tarifa
# con el consumo máximo.
df_per2["Bool"] = 0
df_dummy = df_per2.copy()
for i in df_per2.itertuples():
    if ( i.Consumo == max_cons.loc[
        i.CVE_INEGI, "Consumo"] ):
        df_dummy.loc[i[0], "Bool"] = 1

# Quita las tarifas que no
# tengan el consumo máximo.
df_dummy = ( df_dummy.where(df_dummy["Bool"] == 1)
    .drop("Bool", axis = 1).dropna() )

df_per2 = df_dummy.copy()

# Revisa que tengamos tantas
# columnas como municipios.
df_dummy.shape

(2469, 8)

In [ ]:
# Escoge únicamente la tarifa DAC.
df_dac = df_per[df_per["Tarifa"] == "DAC"].copy()

# Revisa los municipios con mayor
# porcentaje de consumo DAC.
df_dac.sort_values("Consumo", ascending = False).iloc[0:85]#17

,CVE_ENT,CVE_MUN,CVE_INEGI,NOM_ENT,NOM_MUN,Tarifa,Consumo,Usuarios
44737,19,19,19019,Nuevo Leon ...,San Pedro Garza Garcia ...,DAC,0.416427,0.148404
12742,9,4,9004,Distrito Federal ...,Cuajimalpa de Morelos ...,DAC,0.406100,0.139831
8872,7,111,7111,Chiapas ...,Zinacantan ...,DAC,0.376670,0.106787
13282,9,16,9016,Distrito Federal ...,Miguel Hidalgo ...,DAC,0.352456,0.101009
32137,15,37,15037,Mexico ...,Huixquilucan ...,DAC,0.351611,0.131892
...,...,...,...,...,...,...,...,...
3022,5,30,5030,Coahuila de Zaragoza ...,Saltillo ...,DAC,0.107090,0.034059
33217,15,61,15061,Mexico ...,Nopaltepec ...,DAC,0.106572,0.026086
31282,15,18,15018,Mexico ...,Calimaya ...,DAC,0.105786,0.024455
86722,26,2,26002,Sonora ...,Agua Prieta ...,DAC,0.104263,0.032902


In [ ]:
# Suma el consumo de tarifa 1 por municipio.
df_fin = df_2.copy()
df_fin = df_fin.groupby("CVE_INEGI").sum()

cols = ["CVE_ENT", "CVE_MUN", "NOM_ENT", "NOM_MUN"]

# Copia los datos de cada municipio.
for i in cols:
    df_fin[i] = df.loc[df["Tarifa"] == "1", i].array

# Escoge la tarifa tipo 1 para cada municipio.
df_fin["Tarifa"] = df_per2["Tarifa"].array
# Reordena las columnas.
df_fin = df_fin[ list(df_fin.columns[0:2])
    + list(df_fin.columns[-3:])
    + list(df_fin.columns[2:-3])
    ]
# Convierte el número de
# municipio índice a columna.
df_fin.reset_index(inplace = True)

# Agrega el consumo en tarifa DAC.
df_fin = ( pd.concat( 
    [df_fin, df[df["Tarifa"] == "DAC"]] )
    .sort_values(["CVE_INEGI", "Tarifa"] ) )

# Corrige la tarifa de ciertos municipios 
# que han cambiado recientemente.
df_fin.loc[( (df_fin["CVE_INEGI"] == 8053) &
    (df_fin["Tarifa"] == "1") ), "Tarifa"] = "1C"
df_fin.loc[( (df_fin["CVE_INEGI"] == 8059) &
    (df_fin["Tarifa"] == "1") ), "Tarifa"] = "1A"

# Guarda el archivo.
df_fin.to_csv(path + "Electrico_residencial_2010_2017.csv",
    index = False)